# Value-at-Risk

## Import Libraries

In [30]:

import numpy as np
import pandas as pd
import requests
from dateutil import relativedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Risk Free Rate 
RF = 0.05
CONF_LEVEL = 95
TIME_HORIZEN = 1


## Historical Value-at-Risk

In [4]:
def get_alphavantagekey(path):
    with open(path) as f:
        key  = f.read().strip()
    return key
KEY=get_alphavantagekey('keys/alphavantage.txt')

In [36]:
class Security:
    def __init__(self, name, identifier, key, time_horizen=1, conf_level=95):
        self.name = name
        self.identifier = identifier
        self.conf_level = conf_level
        self.time_horizen = time_horizen
        self.get_historical_price_data(key)
        self.HistVaR = self.calc_HistVaR()
        self.HistCVaR = self.calc_HistCVaR()

    def __str__(self):
        """Print Security Attributes"""

        # calcualte the lenght of historical daily data
        from_ = min(self.prices.index).date()
        to_ = max(self.prices.index).date()
        datediff = relativedelta.relativedelta(to_,from_)

        if datediff.years > 1:
            freq = f"{datediff.years:.0f} Years, {datediff.months:.0f} Months"
        else:
            freq = f"{datediff.days} days"
      
        # print message
        txt = f"{self.name}({self.identifier})\n"
        txt += "="*100 + "\n"
        txt += f"Date Range:{from_} - {to_} ({freq})\n"
        txt += f"Total Return: {self.total_return*100:.2f}%\n"
        txt += f"Annualized Return: {self.mu*100:.2f}% | Annualized Volatility: {self.sigma*100:.2f}%\n"
        txt += f"Historical VaR ({self.time_horizen} days): {self.HistVaR*100:.2f}% | Historical CVaR: {self.HistCVaR*100:.2f}%\n"
        return txt
    
    def calc_HistVaR(self):
        """Historical Value-at-Risk for a confidence interval"""
        return np.percentile(self.returns,100-self.conf_level) * np.sqrt(self.time_horizen)

    def calc_HistCVaR(self):
        """
        Historical Conditional Value-at-Risk for a confidence interval
        Find the average of the worst returns below the VaR
        """
        belowVaR = self.returns <= self.HistVaR
        return self.returns[belowVaR].mean() * np.sqrt(self.time_horizen)

    def generate_returns(self, mu, sigma, n):
        # init an array of n returns from the normal dist with mean=mu and stdev = sigma
        self.returns = np.random.default_rng().normal(mu, sigma, n)

    def get_historical_price_data(self,key):
        """
        Returns daily data for a stock (symbol)
        key: api key
        symbols: VMW,AAPL,GOOG,SPY
        """
        def calc_return(returns):
            """ calculate annualized return from daily returns"""
            return np.prod(1 + returns) ** (252/len(returns)) - 1
        def calc_volatility(returns):
            """calculate annualized volatilit from daily returns"""
            return np.std(returns) * np.sqrt(252)
        def calc_total_return(self):
            """calculate the total return for the period"""
            index_first = self.prices.index[0]
            index_last = self.prices.index[-1]
            return self.prices.loc[index_last,'close'] / self.prices.loc[index_first, 'close'] -1

        # request data from alphavantage
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={self.identifier}&apikey={key}&outputsize=full"
        r = requests.get(url)
        d = r.json()

        # extract data to a df
        df = pd.DataFrame(d['Time Series (Daily)']).T
        df.columns = ['open','high','low','close','volume']
        df['symbol'] = self.identifier

        # change data types
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()

        # convert datatype to float
        for col in ['open','high','low','close','volume']:
            df[col] = df[col].astype('float')

        # calc daily returns
        df['returns'] = df['close'].pct_change()

        # update the security attributes
        self.prices = df
        self.returns = np.array(df['returns'].dropna())
        self.mu = calc_return(self.returns)
        self.sigma = calc_volatility(self.returns)
        self.total_return = calc_total_return(self)
       
    def plot_prices(self):
        df = self.prices
        hover_temp = "Date:%{x}"+ " Closing Price:%{y:,.0f}"
        title = f"{self.name} ({self.identifier}) | Return: {self.mu*100:.2f}% | Volatility: {self.sigma*100:.2f}% | TR:{self.total_return*100:.2f}%"
        fig = go.Figure(
            go.Scatter(
                x = df.index,
                y = df.close,
                line = dict(color = 'rgb(83,128,141)',width=2),
                fill='tozeroy',
                fillcolor = 'rgba(83,128,141,0.5)',
                hovertemplate = hover_temp
            )
        )
        fig.update_layout(
            template='plotly_white',
            title = title,
            yaxis_title='Closing Price' ,
            width=600, height=500
        )
        fig.show()
        
    def plot_histograms(self):
        """"Plot the histogram of daily returns with VaR"""

        title = f"<b>Historical VaR ({self.time_horizen} days) </b><br>VaR:{self.HistVaR*100:.2f}% | CVaR: {self.HistCVaR*100:.2f}%"
        returns = self.returns *100
        min_r = min(self.returns)*100
        max_r = max(self.returns)*100

        # histogram
        fig = go.Figure(
            go.Histogram(
                x = returns,
                marker_color = 'rgba(83,128,141,0.8)'
            )
        )
        # CVaR Area
        fig.add_shape(
            type='rect',
            x0=min_r,
            x1= self.HistVaR*100,
            y0=0,
            y1=1,
            fillcolor='red',
            opacity=0.1
        )
        # Non-CVaR Area
        fig.add_shape(
            type='rect',
            x1=max_r,
            x0= self.HistVaR*100,
            y0=0,
            y1=1,
            fillcolor='skyblue',
            opacity=0.1
        )
        # VaR threshold
        fig.add_shape(
            type='line',
            x1=self.HistVaR *100,
            x0= self.HistVaR*100,
            y0=0,
            y1=1,
            line=dict(color='rgba(255,51,51,.5)'),
        )

        # VaR Threshold
        fig.add_annotation(
            x=self.HistVaR*100,
            y=0.8,
            xref='x',
            yref='paper',
            text = "VaR<br>Threshold",
            ax=50,
            font = dict(size=14,color='rgba(255,51,51,0.8)'),
            bordercolor = 'rgba(255,51,51,.2)',
            bgcolor = 'rgba(255,51,51,.2)',
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4
        )
        # CVaR Area
        fig.add_annotation(
            x=self.HistVaR*100,
            y=0.5,
            xref='x',
            yref='paper',
            text = "CVaR<br>Area",
            ax=-40,
            font = dict(size=14,color='rgba(255,51,51,0.8)'),
            arrowcolor = 'rgba(255,51,51,.5)',
            showarrow=True,
            arrowhead=4
        )
        fig.update_layout(template = 'plotly_white',width=600, height=500, title=title)
        fig.update_shapes(dict(xref='x',yref='paper'))
        fig.show()


In [88]:
s = Security('S&P500','SPY',KEY, TIME_HORIZEN, CONF_LEVEL)
print(s)
s.plot_prices()
s.plot_histograms()


## Monte Carlo Value-at-Risk

## Variance/Covariance Value-at-Risk

## Conditional Value-at-Risk